In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.document_loaders import DirectoryLoader
import magic
import os
import nltk
from io import StringIO

In [3]:
import numpy as np

In [4]:
documents=[]
relevant='./data/relevant/'
generic='./data/generic'

for text_file in os.listdir(relevant):
    with open(os.path.join(relevant,text_file), 'r') as File:
        documents.append(File.read())
for text_file in os.listdir(generic):
    with open(os.path.join(generic,text_file), 'r') as File:
        documents.append(File.read())

In [5]:
text_splitter=CharacterTextSplitter(chunk_size=256,chunk_overlap=0)

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
model='bert-large-uncased'
model_kwargs={'device':'cuda'}
hf=HuggingFaceEmbeddings(
    model_name=model,
    model_kwargs=model_kwargs
)

No sentence-transformers model found with name /home/prarabdh/.cache/torch/sentence_transformers/bert-large-uncased. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /home/prarabdh/.cache/torch/sentence_transformers/bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification 

In [8]:
texts=text_splitter.create_documents(documents)

Created a chunk of size 439, which is longer than the specified 256
Created a chunk of size 997, which is longer than the specified 256
Created a chunk of size 2735, which is longer than the specified 256
Created a chunk of size 288, which is longer than the specified 256
Created a chunk of size 1233, which is longer than the specified 256
Created a chunk of size 422, which is longer than the specified 256
Created a chunk of size 1102, which is longer than the specified 256
Created a chunk of size 1118, which is longer than the specified 256
Created a chunk of size 1371, which is longer than the specified 256
Created a chunk of size 797, which is longer than the specified 256
Created a chunk of size 1413, which is longer than the specified 256
Created a chunk of size 681, which is longer than the specified 256
Created a chunk of size 1171, which is longer than the specified 256
Created a chunk of size 812, which is longer than the specified 256
Created a chunk of size 1554, which is lo

In [9]:
docsearch = Chroma.from_documents(texts,hf)

Using embedded DuckDB without persistence: data will be transient


In [10]:
qa= VectorDBQA.from_chain_type(llm= OpenAI(), chain_type='stuff', vectorstore= docsearch)

/home/prarabdh/anaconda3/envs/nlp/lib/python3.9/site-packages/langchain/chains/retrieval_qa/base.py:201: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


## ChatBot: BERT Version

In [11]:
from IPython.display import display
import ipywidgets as widgets

In [12]:
from langchain.chains import ConversationalRetrievalChain

In [13]:
qa=ConversationalRetrievalChain.from_llm(OpenAI(temperature=0.1), docsearch.as_retriever())

In [14]:
chat_history=[]
def on_submit(_):
    query=input_box.value
    input_box.value=""

    if query.lower()=="exit":
        print("Exiting.....thanks :)")
        return 
    result=qa({"question":query, "chat_history":chat_history})
    chat_history.append((query,result['answer']))

    display(widgets.HTML(f'<b>User: </b> {query}'))
    display(widgets.HTML(f'<b><font color="blue">Chatbot:</font></b> {result["answer"]}'))
print("Welcome to the Shakespeare chatbot! Type 'exit' to come back to the real world! ")

input_box=widgets.Text(placeholder='Hark! Pose thy query forthwith! ')
input_box.on_submit(on_submit)
display(input_box)

Welcome to the Shakespeare chatbot! Type 'exit' to come back to the real world! 


/tmp/ipykernel_3047242/2865146757.py:17: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  input_box.on_submit(on_submit)


Text(value='', placeholder='Hark! Pose thy query forthwith! ')

HTML(value="<b>User: </b> Who was Shakespeare's wife?")

HTML(value='<b><font color="blue">Chatbot:</font></b>  Shakespeare\'s wife was Anne Hathaway.')

...

HTML(value='<b><font color="blue">Chatbot:</font></b>  Antonio appears in The Merchant of Venice, Twelfth Nigh…

HTML(value="<b>User: </b> Was Shakespeare's age when he died?")

HTML(value='<b><font color="blue">Chatbot:</font></b>  Shakespeare died in his home town of Stratford on 23 Ap…

HTML(value='<b>User: </b> How big is the Universe?')

HTML(value='<b><font color="blue">Chatbot:</font></b>  The size of the Universe is unknown, as we cannot obser…